In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
data_dir = "/kaggle/input/player-teams-classification/dataset_pemain"
image_size = 224
batch_size = 32
num_epochs = 50
patience = 40
checkpoint_path = "best_mobilenetv2_model.pth"

In [ ]:
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    
    # Rotasi ekstrim dan mirror
    transforms.RandomHorizontalFlip(p=1.0),
    transforms.RandomVerticalFlip(p=0.7),
    transforms.RandomRotation(degrees=90),
    
    # Distorsi geometris dan skala ekstrem
    transforms.RandomAffine(
        degrees=60,
        translate=(0.3, 0.3),
        scale=(0.5, 1.5),
        shear=45
    ),
    
    # Warna benar-benar diacak
    transforms.ColorJitter(
        brightness=0.8,
        contrast=0.8,
        saturation=0.8,
        hue=0.3
    ),
    
    # Blur dan noise
    transforms.GaussianBlur(kernel_size=(7, 7), sigma=(2.0, 5.0)),
    transforms.RandomGrayscale(p=1.0),
    
    # Konversi ke tensor dan normalisasi
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])


dataset = datasets.ImageFolder(root=data_dir, transform=transform)
class_names = dataset.classes
num_classes = len(class_names)

train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(
    dataset, [train_size, val_size, test_size]
)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [ ]:
model = models.mobilenet_v2(pretrained=True)
model.classifier[1] = nn.Linear(model.last_channel, num_classes)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
print(f"Train samples: {len(train_dataset)}")
print(f"Val samples: {len(val_dataset)}")
print(f"Test samples: {len(test_dataset)}")

In [ ]:
import time

best_val_loss = float('inf')
patience_counter = 0

train_acc_history, val_acc_history = [], []
train_loss_history, val_loss_history = [], []
epoch_times = []

for epoch in range(num_epochs):
    start_time = time.time()

    # Training
    model.train()
    train_loss, train_correct = 0.0, 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        train_correct += (preds == labels).sum().item()

    train_acc = train_correct / len(train_dataset)
    train_loss_avg = train_loss / len(train_loader)

    # Validation
    model.eval()
    val_loss, val_correct = 0.0, 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            val_correct += (preds == labels).sum().item()

    val_acc = val_correct / len(val_dataset)
    val_loss_avg = val_loss / len(val_loader)

    # Logging
    train_acc_history.append(train_acc)
    val_acc_history.append(val_acc)
    train_loss_history.append(train_loss_avg)
    val_loss_history.append(val_loss_avg)

    elapsed_time = time.time() - start_time
    epoch_times.append(elapsed_time)

    print(f"Epoch {epoch+1}/{num_epochs} | "
          f"Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f} | "
          f"Train Loss: {train_loss_avg:.4f} | Val Loss: {val_loss_avg:.4f} | "
          f"Time: {elapsed_time:.2f}s")

    # Early stopping
    if val_loss_avg < best_val_loss:
        best_val_loss = val_loss_avg
        patience_counter = 0
        print("✅ Validation loss improved.")
    else:
        patience_counter += 1
        print(f"⚠️No improvement. Patience: {patience_counter}/{patience}")
        if patience_counter >= patience:
            print("⛔ Early stopping triggered!")
            break

# Setelah loop selesai
avg_epoch_time = sum(epoch_times) / len(epoch_times)
print(f"\n📊 Rata-rata waktu per epoch: {avg_epoch_time:.2f} detik")


In [ ]:
model.eval()

y_true, y_pred = [], []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        y_true.extend(labels.numpy())
        y_pred.extend(preds.cpu().numpy())

print(classification_report(y_true, y_pred, target_names=class_names))

In [ ]:
plt.figure(figsize=(12,5))

plt.subplot(1,2,1)
plt.plot(train_loss_history, label='Train Loss')
plt.plot(val_loss_history, label='Val Loss')
plt.title('Loss per Epoch')
plt.legend()

plt.subplot(1,2,2)
plt.plot(train_acc_history, label='Train Accuracy')
plt.plot(val_acc_history, label='Val Accuracy')
plt.title('Accuracy per Epoch')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(train_loss_history, label='Train Loss')
plt.plot(val_loss_history, label='Val Loss')
plt.legend()
plt.title('Loss per Epoch')

plt.subplot(1,2,2)
plt.plot(train_acc_history, label='Train Accuracy')
plt.plot(val_acc_history, label='Val Accuracy')
plt.legend()
plt.title('Accuracy per Epoch')
plt.tight_layout()
plt.show()

In [ ]:
import torch
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Pastikan model dalam mode evaluasi
model.eval()

# Untuk menyimpan hasil
y_true = []
y_pred = []

# Jangan hitung gradien saat evaluasi (hemat memori dan lebih cepat)
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, preds = torch.max(outputs, 1)

        y_true.extend(labels.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())

# Hitung akurasi
accuracy = np.mean(np.array(y_true) == np.array(y_pred))
print(f"Akurasi pada test set: {accuracy:.4f}")


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Tampilkan confusion matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()

# Print classification report (precision, recall, f1-score)
print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=class_names))
